# Analisis Sentimen Opini Masyarakat Terhadap Vaksinasi Nasional Menggunakan Metode Naïve Bayes dengan Seleksi Fitur TF-IDF (Bagian Naive Bayes)

## Import library yang dibutuhkan

In [1]:
#!pip install sastrawi
#!pip install sklearn

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from matplotlib import rc
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import csv
import ast
import matplotlib as mpl

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## Memuat data training
Dalam Naive Bayes, kita kudu buat model dari Naive Bayesnya dulu (maksudnya kita kudu buat probabilitas dari tiap kata di tiap kelasnya). Jadi kita muat dulu train set-nya

In [3]:
train = pd.read_csv("csv files/train_80.csv")
train.head()

,label,tweet
0,1,"['vaksin', 'rakyat', 'indonesia', 'mari', 'awa..."
1,0,"['vaksin', 'astrazeneca', 'rekomendasi', 'bpom..."
2,1,"['didik', 'kemenag', 'jombang', 'vaksinasi', '..."
3,0,"['titer', 'antibodi', 'efektifitas', 'acu', 'b..."
4,0,"['vonis', 'mati', 'bpom', 'vaksin', 'nusantara..."


## Memuat isi dari data training
Karena kita udah melakukan preprocessing terhadap semua tweet, jadi kita tinggal memuat hasil tokenisasi yang ada di data training aja.

In [4]:
class Tokenizer:
    def tokenize(self, texts):
        texts = ast.literal_eval(texts)
        return [text for text in texts]

## Naive Bayes!
Ini class dan function paling penting, karena didalam class ini terdapat function untuk mendapatkan hasil prediksi dengan menggunakan Naive Bayes. Inget tahapannya:
- Ambil data training yang udah ngelewatin preprocessing
- Ambil fitur-fitur yang dibutuhkan Naive Bayes berdasarkan perangkingan TF-IDF dengan persentase yang ditentukan (80%)
- Tiap kata yang ada didalam tweet kita buat tabel kemunculannya (berdasarkan tweet dan sentimennya)
- Hitung probabilitas dari tiap kata dalam tiap kelas dan probabilitas priornya
- Prediksi data test dengan model yang udah dibuat!
- Hitung Akurasi, Presisi, dan Recall/Sensitivity
Btw, yang ada dibawah ini cuma class. Artinya kodingan dibawah ini cuma blueprint-nya. Kita akan "run" kodingannya nanti.

In [5]:
class MultinomialNaiveBayes:
    def __init__(self, classes, tokenizer):
        self.tokenizer = tokenizer
        self.classes = classes
        
    # Di function ini, kita pisahin tweet mana yang sentimennya negatif dan mana yang sentimennya positif  
    def group_by_class(self, X, y):
        data = dict()
        for c in self.classes:
            data[c] = X[np.where(y == c)]
        return data
    
    # function ini untuk mendapatkan fitur yang udah didapetin dari hasil perangkingan TF-IDF
    def get_features(self, features_percentage):
        all_features = []
        nb_features = []
        with open("csv files/tfidf_rank_train80.csv", newline='') as inputfile:
            for row in csv.reader(inputfile):
                all_features.append(row[0])
        i = 0
        while i < int(len(all_features)*features_percentage): 
            nb_features.append(all_features[i])
            i = i + 1
        return nb_features
    
    # Function ini untuk ngebuat model Naive Bayes, atau bisa dibilang untuk dapetin probabilitas dari tiap kata terhadap sentimennya
    def fit(self, p, X, y):
        self.n_class_items = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set(self.get_features(p))

        n = len(X)
        
        grouped_data = self.group_by_class(X, y)
        
        for c, data in grouped_data.items():
            self.n_class_items[c] = len(data)
            self.log_class_priors[c] = math.log(self.n_class_items[c] / n)
            self.word_counts[c] = defaultdict(lambda: 0)
          
            for text in data:
                counts = Counter(self.tokenizer.tokenize(text))
                for word, count in counts.items():
                    if word in self.vocab:
                        self.word_counts[c][word] += count
        return self
    
    # Ini buat laplace smoothing, menambahkan satu ke pembilang
    def laplace_smoothing(self, word, text_class):
        num = self.word_counts[text_class][word] + 1
        denom = self.n_class_items[text_class] + len(self.vocab)
        return math.log(num / denom)
    
    # lalu ini buat prediksi
    def predict(self, X):
        result = []
        for text in X:  
            class_scores = {c: self.log_class_priors[c] for c in self.classes}

            words = set(self.tokenizer.tokenize(text))
            for word in words:
                if word not in self.vocab: continue
                
                for c in self.classes:
                    log_w_given_c = self.laplace_smoothing(word, c)
                    class_scores[c] += log_w_given_c
                
            result.append(max(class_scores, key=class_scores.get))

        return result

## It's training time!
Tadi kan aku udah bilang kalo kodingan diatas itu cuma blueprint aja. Karena function yang ada diatas itu butuh nilai parameter, jadi bisa dibilang kerjaan awal kita itu deklarasiin variabel untuk parameternya dulu. Baru deh abis itu kita prediksi sentimennya!

In [6]:
# Ambil tweet dari yang ingin dimasukkan ke dalam fitur
X_train = train['tweet'].values
# Ambil nilai dari sentimennya
y_train = train['label'].values
# Mau menggunakan fiturnya berapa persen?
fp = 0.8

In [7]:
# Hitung probabilitas dari tiap kata dalam tiap kelas dan hitung probabilitas prior tiap kelas
MNB = MultinomialNaiveBayes(classes=np.unique(y_train), tokenizer=Tokenizer()).fit(fp, X_train, y_train)

### Testing!
Proses diatas kan buat modelnya tuh, artinya kita udah bisa ngitung sentimen analisisnya

In [8]:
test = pd.read_csv("csv files/test_20.csv")
X_test = test['tweet'].values
y_test = test['label'].values

In [9]:
y_pred = MNB.predict(X_test)

In [11]:
print(y_test)

[1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1]


In [13]:
print(y_pred)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
accuracy_score(y_test, y_pred)

0.9682539682539683

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
mpl.rcParams['figure.figsize'] = 14, 8

class_names = ["negative", "positive"]
fig,ax = plt.subplots()


sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="Blues", fmt="d", cbar=False, xticklabels=class_names, yticklabels=class_names)
ax.xaxis.set_label_position('top')
plt.tight_layout()
plt.ylabel('Actual sentiment')
plt.xlabel('Predicted sentiment');